In [1]:
!pip install Kaggle

In [105]:
!pip install nltk
import nltk
nltk.download('punkt_tab')


[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [106]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [101]:
import os
import json
import nltk

from zipfile import ZipFile
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from nltk.corpus import stopwords
from wordcloud import WordCloud,STOPWORDS
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from bs4 import BeautifulSoup
from string import punctuation
from nltk import pos_tag
from nltk.corpus import wordnet
import re, string, unicodedata


**Data** **Extraction**

In [43]:
kaggle_dictionary= json.load(open("kaggle.json"))

In [44]:
os.environ["KAGGLE_USERNAME"]=kaggle_dictionary["username"]
os.environ["KAGGLE_KEY"]=kaggle_dictionary["key"]

In [45]:
!kaggle datasets download -d lakshmi25npathi/imdb-dataset-of-50k-movie-reviews

Dataset URL: https://www.kaggle.com/datasets/lakshmi25npathi/imdb-dataset-of-50k-movie-reviews
License(s): other
imdb-dataset-of-50k-movie-reviews.zip: Skipping, found more recently modified local copy (use --force to force download)


In [46]:
!ls


'IMDB Dataset.csv'   imdb-dataset-of-50k-movie-reviews.zip   kaggle.json   sample_data


In [47]:
#unzipping the dataset
with ZipFile("imdb-dataset-of-50k-movie-reviews.zip") as zip_ref:
  zip_ref.extractall()

In [48]:
!ls

'IMDB Dataset.csv'   imdb-dataset-of-50k-movie-reviews.zip   kaggle.json   sample_data


**Loading** **Dataset**

In [49]:
df=pd.read_csv("IMDB Dataset.csv")

In [50]:
df.shape

(50000, 2)

In [51]:
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [52]:
#class imbalance check
df["sentiment"].value_counts()

,count
sentiment,
positive,25000
negative,25000


In [103]:
stop = set(stopwords.words('english'))
punctuations = list(string.punctuation)
stop.update(punctuations)

In [107]:
# picking random text form review column
sample_text = df.review[8]
sample_text

# Removing HTML
soup = BeautifulSoup(sample_text, "html.parser")
sample_text = soup.get_text()

# Replacing sepcial characters and symbols, non-alphabets with sapces
sample_text = re.sub('\[[^]]*\]',' ',sample_text)
sample_text = sample_text.lower() # converting text into lowercase

sample_text = re.sub('[^a-zA-Z]',' ',sample_text)


# Removing stop words
from nltk.corpus import stopwords

sample_text = nltk.word_tokenize(sample_text)

sample_text = [word for word in sample_text if not word in set(stopwords.words("english"))]


In [108]:
def strip_html(text):
    soup = BeautifulSoup(text, "html.parser")
    return soup.get_text()

#Removing the square brackets
def remove_between_square_brackets(text):
    return re.sub('\[[^]]*\]', '', text)

# Removing URL's
def remove_between_square_brackets(text):
    return re.sub(r'http\S+', '', text)

#Removing the stopwords from text
def remove_stopwords(text):
    final_text = []
    for i in text.split():
        if i.strip().lower() not in stop and i.strip().lower().isalpha():
            final_text.append(i.strip().lower())
    return " ".join(final_text)

#Removing the noisy text
def denoise_text(text):
    text = strip_html(text)
    text = remove_between_square_brackets(text)
    text = remove_stopwords(text)
    return text

# Applying function on review column
df['review'] = df['review'].apply(denoise_text)

<ipython-input-108-65eff986ee0b>:2: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(text, "html.parser")


In [109]:
df.replace({"sentiment":{"positive":1, "negative":0}}, inplace=True)

In [110]:
#split train and test data
train_data, test_data= train_test_split(df, test_size=0.2, random_state=42)

In [111]:
train_data.shape

(40000, 2)

In [112]:
test_data.shape

(10000, 2)

**Data** **Preprocessing**

In [113]:
#Tokenization
tokenizer= Tokenizer(num_words=5000)
tokenizer.fit_on_texts(train_data["review"])
x_train= pad_sequences(tokenizer.texts_to_sequences(train_data["review"]), maxlen=200)
x_test= pad_sequences(tokenizer.texts_to_sequences(test_data["review"]), maxlen=200)

In [115]:
print(x_train)

[[   0    0    0 ...  349   87  185]
 [   0    0    0 ...    1   18   28]
 [   0    0    0 ...  316  187  583]
 ...
 [   0    0    0 ...  345 1072 1568]
 [   0    0    0 ...   22 1086   28]
 [   0    0    0 ...   41  347   11]]


In [116]:
print(x_test)

[[   0    0    0 ...  320  454  727]
 [   0    0    0 ...   28 1765    9]
 [   0    0    0 ...   34  723  837]
 ...
 [   0    0    0 ... 1988  375  113]
 [   0    0    0 ...  555  160   61]
 [   0    0    0 ...    7    9  198]]


In [117]:
y_train= train_data["sentiment"]
y_test= test_data["sentiment"]


**Building** **LSTM** **model**

In [118]:
 model= Sequential()
 model.add(Embedding(input_dim=5000, output_dim=128, input_length=200))
 model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
 model.add(Dense(1, activation="sigmoid"))
 input_shape = (None, 200) # (batch_size, input_length)
 model.build(input_shape=input_shape)

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [119]:
model.summary()

Model: "sequential_19"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_19 (Embedding)             │ (None, 200, 128)            │         640,000 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_17 (LSTM)                       │ (None, 128)                 │         131,584 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_17 (Dense)                     │ (None, 1)                   │             129 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 771,713 (2.94 MB)

 Trainable params: 771,713 (2.94 MB)

 Non-trainable params: 0 (0.00 B)

In [120]:
#compile the model
model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

**Training** **the** **model**

In [121]:
from tensorflow.keras.callbacks import EarlyStopping
earlystop= EarlyStopping(monitor="val_loss", patience=5, restore_best_weights=True)

In [122]:
history= model.fit(x_train, y_train, batch_size=64, epochs=5, validation_split=0.2, callbacks=[earlystop])

Epoch 1/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 252s 500ms/step - accuracy: 0.7537 - loss: 0.5045 - val_accuracy: 0.8574 - val_loss: 0.3404
Epoch 2/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 272s 543ms/step - accuracy: 0.8750 - loss: 0.3091 - val_accuracy: 0.8599 - val_loss: 0.3351
Epoch 3/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 302s 504ms/step - accuracy: 0.8952 - loss: 0.2651 - val_accuracy: 0.8590 - val_loss: 0.3460
Epoch 4/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 249s 499ms/step - accuracy: 0.9084 - loss: 0.2265 - val_accuracy: 0.8561 - val_loss: 0.3597
Epoch 5/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 249s 499ms/step - accuracy: 0.9220 - loss: 0.2042 - val_accuracy: 0.8519 - val_loss: 0.3709


**Model** **Evaluation**

In [123]:
loss, accuracy= model.evaluate(x_test, y_test)
print("Test Loss:", loss)
print("Test Accuracy:", accuracy)

313/313 ━━━━━━━━━━━━━━━━━━━━ 37s 116ms/step - accuracy: 0.8568 - loss: 0.3343
Test Loss: 0.33515414595603943
Test Accuracy: 0.8567000031471252


In [124]:
model.save("sentiment_analysis_model_lstm.h5")

In [125]:
def prediction_sentiment(review):
  sequence= tokenizer.texts_to_sequences([review])
  padded_sequence= pad_sequences(sequence, maxlen=200)
  prediction= model.predict(padded_sequence)
  sentiment= "positive" if prediction > 0.5 else "negative"
  return sentiment
